In [59]:
from typing import List

import pandas as pd
import statsmodels.api as sm
from scipy.stats import ttest_rel

In [61]:
df = pd.read_csv("../data/lalonde_data.csv")

In [62]:
df.head()

,ID,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78
0,NSW1,1,37,11,1,0,1,1,0.0,0.0,9930.0460
1,NSW2,1,22,9,0,1,0,1,0.0,0.0,3595.8940
2,NSW3,1,30,12,1,0,0,0,0.0,0.0,24909.4500
3,NSW4,1,27,11,1,0,0,1,0.0,0.0,7506.1460
4,NSW5,1,33,8,1,0,0,1,0.0,0.0,289.7899


In [63]:
OUTCOME = 're78'
TREAT = 'treat'
PROP_MODEL = 'prop_model_yhat'
VARS = ['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75']

### Question 1

Find the standardized differences for all of the confounding variables (pre-matching). What is the standardized difference for married (to nearest hundredth)?    

In [64]:
def standardized_difference(pre: pd.Series, post: pd.Series) -> float:
    return abs(post.mean() - pre.mean()) / ((post.std() + pre.std()) / 2)

In [65]:
def standardized_difference_all_cols(df: pd.DataFrame, treat_col: str, var_cols: List[str]) -> pd.Series:
    pre_treat_df = df[df[treat_col] == 1].reset_index(drop=True)
    post_treat_df = df[df[treat_col] == 0].reset_index(drop=True)
    
    all_var_std_diff = {}
    for var in var_cols:
        var_std_diff = standardized_difference(pre_treat_df[var], post_treat_df[var])
        all_var_std_diff[var] = var_std_diff
        
    return pd.Series(all_var_std_diff)

In [66]:
standardized_difference_all_cols(df, TREAT, VARS).round(2)

age         0.25
educ        0.05
black       1.67
hispan      0.28
married     0.72
nodegree    0.24
re74        0.60
re75        0.29
dtype: float64

### Question 2

What is the raw (unadjusted) mean of real earnings in 1978 for treated subjects minus the mean of real earnings in 1978 for untreated subjects? 

In [67]:
df.loc[df[TREAT] == 1, OUTCOME].mean() - df.loc[df[TREAT] == 0, OUTCOME].mean()

-635.0262120374209

### Question 3

Fit a propensity score model. Use a logistic regression model, where the outcome is treatment. Include the 8 confounding variables in the model as predictors, with no interaction terms or non-linear terms (such as squared terms). Obtain the propensity score for each subject.

What are the minimum and maximum values of the estimated propensity score?

In [84]:
propensity_model = sm.Logit(df[TREAT], sm.add_constant(df[VARS])).fit()
propensity_model.summary()

Optimization terminated successfully.
         Current function value: 0.397267
         Iterations 7


/Users/giuliano.ferrari/Documents/Other/causality-crash-course/python/.env/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  treat   No. Observations:                  614
Model:                          Logit   Df Residuals:                      605
Method:                           MLE   Df Model:                            8
Date:                Sat, 18 Sep 2021   Pseudo R-squ.:                  0.3508
Time:                        20:39:33   Log-Likelihood:                -243.92
converged:                       True   LL-Null:                       -375.75
Covariance Type:            nonrobust   LLR p-value:                 2.194e-52
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.7286      1.017     -4.649      0.000      -6.722      -2.735
age            0.0158      0.014      1.162      0.245      -0.011       0.042
educ           0.1613      0.065      2.477      0.013       0.034       0.289
black          3.0654      0.287     10.698      0.000       2.504       3.627
hispan         0.9836      0.426      2.311      0.021       0.149       1.818
married       -0.8321      0.290     -2.866      0.004      -1.401      -0.263
nodegree       0.7073      0.338      2.095      0.036       0.045       1.369
re74       -7.178e-05   2.87e-05     -2.497      0.013      -0.000   -1.54e-05
re75        5.345e-05   4.63e-05      1.153      0.249   -3.74e-05       0.000
==============================================================================
"""

In [87]:
df[PROP_MODEL] = propensity_model.predict(sm.add_constant(df[VARS]))

/Users/giuliano.ferrari/Documents/Other/causality-crash-course/python/.env/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [88]:
df[PROP_MODEL].min(), df[PROP_MODEL].max()

(0.009080192945777365, 0.8531528452105871)

### Question 4

Now carry out propensity score matching using the Match function. 

Before using the Match function, first do:

```
set.seed(931139)
```

Setting the seed will ensure that you end up with a matched data set that is the same as the one used to create the solutions.

Use options to specify pair matching, without replacement, no caliper. 

Match on the propensity score itself, not logit of the propensity score. Obtain the standardized differences for the matched data.

What is the standardized difference for married?     

In [164]:
from sklearn.utils import shuffle


In [185]:
def pair_match(df: pd.DataFrame, score_col: str, treat_col: str) -> pd.DataFrame:
    treated_df = shuffle(df[df[treat_col] == 1]).reset_index(drop=True)
    not_treated_df = shuffle(df[df[treat_col] == 0]).reset_index(drop=True)
    full_df = treated_df.copy()

    for score in treated_df[score_col].values:        
        idxmin = (score - not_treated_df[score_col]).abs().idxmin()
        full_df = full_df.append(not_treated_df.loc[idxmin], ignore_index=True)
        not_treated_df = not_treated_df.drop(index=[idxmin])
    
    return full_df

In [186]:
matched_df = pair_match(df, PROP_MODEL, TREAT)
standardized_difference_all_cols(matched_df, TREAT, VARS).round(3)

age         0.058
educ        0.111
black       0.862
hispan      0.482
married     0.054
nodegree    0.150
re74        0.054
re75        0.028
dtype: float64

### Question 6

Re-do the matching, but use a caliper this time. Set the caliper=0.1 in the options in the Match function.

Again, before running the Match function, set the seed:

```
set.seed(931139)
```

How many matched pairs are there? 

In [108]:
def pair_match(df: pd.DataFrame, score_col: str, treat_col: str, caliper=None) -> pd.DataFrame:
    treated_df = df[df[treat_col] == 1].sample(frac=1.).reset_index(drop=True)
    not_treated_df = df[df[treat_col] == 0].sample(frac=1.).reset_index(drop=True)
    full_df = pd.DataFrame({})

    for i, row in treated_df.iterrows():
        score_diffs = (row[score_col] - not_treated_df[score_col]).abs()
        if caliper and score_diffs.min() > caliper:
            continue
        idxmin = score_diffs.idxmin()
        full_df = full_df.append(row, ignore_index=True)
        full_df = full_df.append(not_treated_df.loc[idxmin], ignore_index=True)
        not_treated_df = not_treated_df.drop(index=[idxmin])
    
    return full_df

In [109]:
matched_caliper_df = pair_match(df, PROP_MODEL, TREAT, .1)

In [112]:
len(matched_caliper_df)

228

### Question 7

Use the matched data set (from propensity score matching with caliper=0.1) to carry out the outcome analysis. 

For the matched data, what is the mean of real earnings in 1978 for treated subjects minus the mean of real earnings in 1978 for untreated subjects? 

In [113]:
matched_caliper_df.loc[matched_caliper_df[TREAT] == 1, OUTCOME].mean() - matched_caliper_df.loc[matched_caliper_df[TREAT] == 0, OUTCOME].mean()

1747.5399390350858

### Question 8

Use the matched data set (from propensity score matching with caliper=0.1) to carry out the outcome analysis.

Carry out a paired t-test for the effect of treatment on earnings. What are the values of the 95% confidence interval?

In [60]:
ttest_rel(matched_caliper_df.loc[matched_caliper_df[TREAT] == 1, OUTCOME], matched_caliper_df.loc[matched_caliper_df[TREAT] == 0, OUTCOME])

Ttest_relResult(statistic=1.6129600517609903, pvalue=0.10954255057673892)